Find metastic samples and map to sample_id

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option('display.expand_frame_repr', False)

In [27]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import pickle

## Get sample_id of metastatics

In [3]:
db = MongoClient()['tcga']['masked_v2']

In [4]:
samples = []

for sam in db.find():
    samples.append([sam['file_id'], sam['sample_type'], sam['project_id']])

In [5]:
dd = pd.DataFrame(samples, columns=['id', 'type', 'project'])

In [17]:
outpath = '/Users/bogao/DataFiles/new landscape/files/metastatic_'

Metastatic from SKCM

In [13]:
dd.loc[(dd.type=='Metastatic') & (dd.project=='TCGA-SKCM'), 'id'].to_csv(outpath+'skcm.txt', index=False)

/Users/bogao/anaconda3/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


Metastatic from other projects

In [22]:
dd.loc[(dd.type=='Metastatic') & (dd.project!='TCGA-SKCM'), ['id', 'project']].to_csv(outpath+'other.csv', index=False)

In [23]:
pc = MongoClient()['pcawg']['samples_hg38']

In [24]:
pc_samples = []
for sam in pc.find():
    pc_samples.append([sam['sample_id'], sam['sample_type'], sam['project_code']])

In [25]:
dpc = pd.DataFrame(pc_samples, columns=['id', 'type', 'project'])

In [28]:
dpc.loc[(dpc.type=='Metastatic tumour') & (dpc.project=='MELA-AU'), 'id'].to_csv(outpath+'mela-au.txt', index=False)

/Users/bogao/anaconda3/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


In [29]:
dpc.loc[(dpc.type=='Metastatic tumour') & (dpc.project=='PRAD-UK'), 'id'].to_csv(outpath+'prad-uk.txt', index=False)

/Users/bogao/anaconda3/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


## Make a dataset of metastatics

In [6]:
filepath = '/Users/bogao/DataFiles/new landscape/files/metastatic_skcm.txt'
met_skcm = np.genfromtxt(filepath, dtype=str)

In [12]:
filepath = '/Users/bogao/DataFiles/new landscape/files/metastatic_other.csv'
met_others = pd.read_csv(filepath)

In [14]:
filepath = '/Users/bogao/DataFiles/new landscape/files/metastatic_mela-au.txt'
met_mela_au= np.genfromtxt(filepath, dtype=str)

In [16]:
filepath = '/Users/bogao/DataFiles/new landscape/files/metastatic_prad-uk.txt'
met_prad_uk= np.genfromtxt(filepath, dtype=str)

In [19]:
met_ids = np.concatenate((met_skcm, met_others['id'].values, met_mela_au, met_prad_uk))

In [21]:
met_ids.shape

(499,)

In [22]:
met_skcm.shape

(368,)

In [23]:
met_others.shape

(26, 2)

In [24]:
met_mela_au.shape

(60,)

In [25]:
met_prad_uk.shape

(45,)

In [28]:
filepath = '/Users/bogao/DataFiles/new landscape/data/all_bands_meta.pkl'
with open(filepath, 'rb') as fin:
    bands_meta = pickle.load(fin)

In [31]:
bands_meta

,id,source,project,morphology,topography,num_segs,site
0,440e9ec5-8e61-4f75-b1d1-616941d9456d,TCGA,TCGA-OV,84413,Ovary,391,Ovary
1,6ca6f0fe-8b53-421e-ad4e-dfbf8c8c5b59,TCGA,TCGA-OV,84413,Ovary,571,Ovary
2,da9af00f-60ea-456b-9f25-0ab560962769,TCGA,TCGA-OV,84413,Ovary,431,Ovary
3,8e84a028-5e9c-4d01-9791-db0f04b05bce,TCGA,TCGA-OV,84603,Ovary,394,Ovary
4,d3f8c91f-e4cc-4018-9fbf-4368dd036b01,TCGA,TCGA-OV,84413,Ovary,252,Ovary
...,...,...,...,...,...,...,...
42815,PGX_AM_BS_GSM1414445,arraymap,GSE58579,84421,c56.9,104,Ovary
42816,PGX_AM_BS_GSM1414446,arraymap,GSE58579,84421,c56.9,240,Ovary
42817,PGX_AM_BS_GSM1414451,arraymap,GSE58579,84421,c56.9,79,Ovary
42818,PGX_AM_BS_GSM1431034,arraymap,GSE58579,84421,c56.9,133,Ovary


In [32]:
bands_meta[bands_meta.id == met_ids[0]]

,id,source,project,morphology,topography,num_segs,site
608,d13dec04-195e-4470-9872-3b8094296d7b,TCGA,TCGA-SKCM,87203,"Skin, NOS",141,Skin


In [41]:
bands_meta[np.isin(bands_meta.id, met_ids)]

,id,source,project,morphology,topography,num_segs,site
339,54236196-de21-4f37-9e75-0442d03c9d8e,TCGA,TCGA-BRCA,85003,"Breast, NOS",416,Breast
375,f84b79f1-bc6e-4d52-9cd9-12f0dcfee04b,TCGA,TCGA-BRCA,85203,"Breast, NOS",229,Breast
608,d13dec04-195e-4470-9872-3b8094296d7b,TCGA,TCGA-SKCM,87203,"Skin, NOS",141,Skin
609,5ec990df-236e-496b-943d-421223ba9f19,TCGA,TCGA-SKCM,87203,"Skin, NOS",409,Skin
611,8f598ed3-c297-4856-90b8-811c56381ca6,TCGA,TCGA-SKCM,87203,"Skin, NOS",160,Skin
...,...,...,...,...,...,...,...
11575,30600994-c1ec-44c9-a620-3f6c617eb928,PCAWG,MELA-AU,C43.4,c43.4,250,Skin
11582,0bfd1068-3fca-a95b-e050-11ac0c4860c3,PCAWG,PRAD-UK,C61,c61,107,Prostate
11612,0c0038ff-6cc4-b0b0-e050-11ac0d483d73,PCAWG,PRAD-UK,C61,c61,426,Prostate
11635,0bfebf9f-c77f-e57d-e050-11ac0d487827,PCAWG,PRAD-UK,C61,c61,417,Prostate


In [42]:
matfile = '/Users/bogao/DataFiles/new landscape/data/selected_gene_mat.pkl'
labelfile = '/Users/bogao/DataFiles/new landscape/data/all_bands_label.pkl'
dlabelfile = '/Users/bogao/DataFiles/new landscape/data/all_bands_disease_label.pkl'

In [43]:
with open(matfile, 'rb') as fmat, open(labelfile, 'rb') as flab, open(dlabelfile, 'rb') as fdl:
        data = pickle.load(fmat)
        data = np.array(data)
        labels = pickle.load(flab)
        d_labels = pickle.load(fdl)

In [44]:
d = data[ np.isin(bands_meta.id, met_ids)]
site = labels [np.isin(bands_meta.id, met_ids)]
disease = d_labels[ np.isin(bands_meta.id, met_ids) ]

In [46]:
np.unique(site, return_counts=True)

(array(['Breast', 'Colon', 'Others', 'Prostate', 'Skin'], dtype=object),
 array([  6,   1,  10,  44, 395]))

In [53]:
data_train = data[ ~np.isin(bands_meta.id, met_ids)]
site_train = labels [~np.isin(bands_meta.id, met_ids)]
disease_train = d_labels[~ np.isin(bands_meta.id, met_ids) ]

In [49]:
data_train.shape

(42364, 3029)

In [50]:
d.shape

(456, 3029)

In [51]:
data_train.shape[0] + d.shape[0]

42820

### site as label

In [54]:
d = data[ labels != 'Others' ]
site = labels [labels != 'Others']
meta = bands_meta[labels != 'Others' ]

In [61]:
data_train = d[ ~np.isin(meta.id, met_ids)]
site_train = site [~np.isin(meta.id, met_ids)]

In [65]:
data_test = d[ np.isin(meta.id, met_ids)]
site_test = site [np.isin(meta.id, met_ids)]

In [68]:
independent_data_site = {'data_train': data_train, 'data_test':data_test, 'site_train':site_train, 'site_test':site_test}

In [69]:
outfile = '/Users/bogao/DataFiles/new landscape/files/multiclass_independent_data_site.pkl'
with open(outfile, 'wb') as fo:
    pickle.dump(independent_data_site, fo)

### disease as label

In [70]:
d = data[ (d_labels != 'Others') & (d_labels != 'None') ]
disease = d_labels[ (d_labels != 'Others') & (d_labels != 'None') ]
meta = bands_meta[ (d_labels != 'Others') & (d_labels != 'None') ]

In [73]:
unique_disease, unique_count = np.unique(disease, return_counts=True)
selected_disease = unique_disease[ unique_count > 50]

In [74]:
d_sel = d [ np.isin(disease, selected_disease)]
disease_sel = disease [ np.isin(disease, selected_disease)]
meta_sel = meta [ np.isin(disease, selected_disease)]

In [77]:
data_train = d_sel[ ~np.isin(meta_sel.id, met_ids)]
disease_train = disease_sel [~np.isin(meta_sel.id, met_ids)]

In [78]:
data_test = d_sel[ np.isin(meta_sel.id, met_ids)]
disease_test = disease_sel [np.isin(meta_sel.id, met_ids)]

In [83]:
independent_data_disease = {'data_train': data_train, 'data_test':data_test, 'disease_train':disease_train, 'disease_test':disease_test}

In [84]:
outfile = '/Users/bogao/DataFiles/new landscape/files/multiclass_independent_data_disease.pkl'
with open(outfile, 'wb') as fo:
    pickle.dump(independent_data_disease, fo)

In [76]:
d_sel.shape

(21655, 3029)

In [82]:
data_train.shape

(21277, 3029)